In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\HP\Downloads\projec_2024\Final model\updated_data_with_img.csv")


In [3]:
df

,Title,Tag,Review,Address,Mention,Comment,Country,Price,Rating,tags,Governorate,img_link
0,Adventure Alegria ASEG Travel,Tours,14.0,"29, Cite des PTT, Hydra, Algiers, Algiers, Alg...",10 mentions of Algeria,Visiting Algeria is a truly remarkable experie...,Algeria,33.56,1.06,Tours,Algiers,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,Fancyellow Travel Services,Tours,151.0,"Algiers, Algiers Province, Algeria",72 mentions of Algeria,that went above and beyond to provide us with ...,Algeria,52.07,1.60,Tours,Algiers,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,Memorial du Martyr,Monuments & Statues,529.0,"Boulevard Khalifa Oulmane, Algiers, Algiers Pr...",31 mentions of Algeria,and very close to the shopping mall where you ...,Algeria,103.15,3.12,Archaeological tourism,Algiers,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,Cederberg Wilderness Area,for fun,86.0,"Clanwilliam, Western Cape, South Africa",12 mentions of Algeria,So instead we did a couple half day hikes arou...,Algeria,43.29,1.34,for fun,Algiers,NaN
4,Timgad,Ancient Ruins,100.0,"Timgad, Batna Province, Algeria",12 mentions of Algeria,Timgad was my main reason to travel to Algeria...,Algeria,45.18,1.40,Archaeological tourism,Algiers,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
...,...,...,...,...,...,...,...,...,...,...,...,...
28410,Palm Beach Palace Tozeur,Hotels,222.0,"Route Touristique, Tozeur, Tozeur Governorate,...",4 mentions of Tunisia,"Like many hotels in Tunisia, this one is suffe...",Tunisia,61.67,1.89,Hotel,Tozeur,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
28411,Hotel Lac Leman,Hotels,43.0,"Rue Lac Leman, Tunis, Tunis Governorate, Tunisia",2 mentions of Tunisia,This is not the kind of hotels you should book...,Tunisia,37.48,1.17,Hotel,Tunis,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
28412,Hotel Riad Meninx Djerba,Hotels,69.0,"Zone Touristique, Midoun, Djerba Island, Meden...",Unknown,No Comment,Tunisia,40.99,1.28,Hotel,Medenine,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
28413,Floriana Meninx Settemari Club,Hotels,7.0,"Djerba Island, Medenine Governorate, Tunisia",Unknown,No Comment,Tunisia,32.61,1.03,Hotel,Medenine,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import random

# Assuming df is your DataFrame containing the relevant data

# Convert Tag, Review, and Comment columns to string type
df['Tag'] = df['Tag'].astype(str)
df['Review'] = df['Review'].astype(str)
df['Comment'] = df['Comment'].astype(str)

# Feature Engineering: TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Tag'] + ' ' + df['Review'] + ' ' + df['Comment'])

def get_recommendations_with_budget(country, governorates, survey_responses, df, num_days, budget, num_plans=1):
    # Ensure the maximum number of days is not more than 7
    if num_days > 7:
        print("The maximum number of days allowed is 7.")
        return []

    # Filter places that match the user's country and any of the governorates
    filtered_df = df[(df['Country'] == country) & (df['Governorate'].isin(governorates))]

    # Check if there is data available
    if filtered_df.empty:
        print("No data found for the specified country and governorates.")
        return []

    # Initialize list to store recommendations for each plan
    all_recommendations = []

    # Loop to generate multiple plans
    for plan_num in range(1, num_plans + 1):
        # Ensure "Restaurants" are always present in user profile
        user_profile = f"{country} {' '.join(governorates)} {' '.join(survey_responses)}"

        # Transform the user profile using the same TF-IDF vectorizer
        user_profile_vectorized = tfidf_vectorizer.transform([user_profile])

        # Calculate the maximum price per day based on the budget and number of days
        max_price_per_day = budget / num_days

        # Initialize recommendations DataFrame for the current plan
        recommendations_df = pd.DataFrame(columns=['Title', 'Price', 'tags', 'Governorate', 'Day'])

        # Track recommended places to avoid duplicates across days
        recommended_titles = set()

        # Select a hotel for the entire stay (outside the daily loop)
        hotels = filtered_df[filtered_df['tags'].str.lower().str.contains('hotel') & (filtered_df['Price'] <= max_price_per_day)]
        hotel_recommendation = pd.DataFrame()
        hotel_price = 0.0

        if not hotels.empty:
            hotel_recommendation = hotels.sample(1)[['Title', 'Price', 'tags', 'Governorate']]
            recommended_titles.add(hotel_recommendation['Title'].iloc[0])
            hotel_price = hotel_recommendation['Price'].iloc[0] * num_days  # Total hotel price for the stay

        # Loop through each day
        day_counter = 1
        for day in range(num_days):
            # Initialize daily recommendations list
            daily_recommendations = []

            # Filter places for the current governorate
            governorate_df = filtered_df[filtered_df['Governorate'].isin(governorates)]

            # Recommend a restaurant each day
            restaurants = governorate_df[governorate_df['tags'].str.lower().str.contains('restaurant') & (governorate_df['Price'] <= max_price_per_day)]
            if not restaurants.empty:
                restaurant_recommendation = restaurants.sample(1)  # Select 1 restaurant
                for _, row in restaurant_recommendation.iterrows():
                    daily_recommendations.append(row)
                    recommended_titles.add(row['Title'])

            # Shuffle survey responses to vary the order of recommendations
            random.shuffle(survey_responses)

            # Add places related to the survey for each day
            place_recommendations = []
            for response in survey_responses:
                # Get indices of places containing the current survey response
                response_indices = [i for i, tag in enumerate(governorate_df['tags']) if response.lower() in tag.lower()]

                # Exclude places already recommended on previous days
                valid_indices = [idx for idx in response_indices if governorate_df.iloc[idx]['Title'] not in recommended_titles]

                # If there are valid places, select a random one within the budget
                if valid_indices:
                    random_index = random.choice(valid_indices)
                    recommendation = governorate_df.iloc[random_index][['Title', 'Price', 'tags', 'Governorate']]
                    place_recommendations.append(recommendation)
                    recommended_titles.add(recommendation['Title'])

            # Limit the number of place recommendations to 1 or 2 based on availability
            num_additional_recommendations = min(2, len(place_recommendations))
            daily_recommendations.extend(place_recommendations[:num_additional_recommendations])

            # Add the daily recommendations to the overall recommendations DataFrame
            for recommendation in daily_recommendations:
                recommendation['Day'] = day_counter
                recommendations_df = pd.concat([recommendations_df, pd.DataFrame([recommendation])])

            day_counter += 1

        # Calculate total plan price including hotel for the current plan
        total_plan_price = hotel_price
        for idx, row in recommendations_df.iterrows():
            total_plan_price += row['Price']

        # Initialize additional_amount_needed
        additional_amount_needed = 0.0

        # Check if the budget is sufficient for the current plan
        if total_plan_price > budget:
            additional_amount_needed = total_plan_price - budget

        # Store recommendations_df in all_recommendations list
        all_recommendations.append((hotel_recommendation, recommendations_df, total_plan_price, additional_amount_needed))

    return all_recommendations

# Example: Get recommendations with multiple plans
user_country = "Egypt"
user_governorates = ["Cairo"]  # Example with a single governorate
user_survey_responses = ["Tours", "Natural views", "Archaeological tourism", "for fun", "Religious Tourism", "parks", "Museums", "malls", "games", "Natural views", "water places"]
num_days = int(input("Enter the number of days you want to spend (up to 7): "))
budget = float(input("Enter your budget: "))

# Check if the number of days is within the allowed limit
if num_days > 7:
    print("The maximum number of days allowed is 7.")
else:
    num_plans = int(input("Enter the number of plans to generate: "))

    recommendations = get_recommendations_with_budget(user_country, user_governorates, user_survey_responses, df, num_days, budget, num_plans)

    # Example of accessing recommendations for each plan
    for plan_num, (hotel, plan, total_price, additional_amount_needed) in enumerate(recommendations, start=1):
        print(f"\n{'='*40}")
        print(f"Plan {plan_num}: You are staying at the hotel: {hotel['Title'].iloc[0]}")
        print(f"Hotel price per day: {hotel['Price'].iloc[0]}")
        print(f"Total hotel price for {num_days} days: {hotel['Price'].iloc[0] * num_days}")

        print(f"\nPlan {plan_num} Recommendations:")
        for day in range(1, num_days + 1):
            print(f"\nDay {day}:")
            day_df = plan[plan['Day'] == day]
            for idx, row in day_df.iterrows():
                tag_name = row['tags']  # Assuming 'tags' contains the tag name
                print(f"{tag_name}, {row['Title']} , Price: {row['Price']}")

        print(f"\nTotal plan price for Plan {plan_num} including hotel: {total_price}")
        if total_price > budget:
            print(f"\nYour budget of {budget} is not sufficient for Plan {plan_num} total plan price of {total_price}.")
            print(f"You need an additional amount of {additional_amount_needed}.")

    # Let the user choose a plan
    selected_plan_num = int(input(f"\nEnter the number of the plan you want to choose (1 to {num_plans}): "))

    # Display the selected plan
    selected_plan = recommendations[selected_plan_num - 1]
    hotel, plan, total_price, additional_amount_needed = selected_plan

    print(f"\n{'='*40}")
    print(f"\nYou have selected Plan {selected_plan_num}.")
    print(f"You will be staying at the hotel: {hotel['Title'].iloc[0]}")
    print(f"Total plan price: {total_price}")
    if total_price > budget:
        print(f"You need an additional amount of {additional_amount_needed}.")

    print("\nSelected Plan Recommendations:")
    for day in range(1, num_days + 1):
        print(f"\nDay {day}:")
        day_df = plan[plan['Day'] == day]
        for idx, row in day_df.iterrows():
            tag_name = row['tags']  # Assuming 'tags' contains the tag name
            print(f"{tag_name}, {row['Title']} , Price: {row['Price']}")


Enter the number of days you want to spend (up to 7): 5
Enter your budget: 1000
Enter the number of plans to generate: 2

Plan 1: You are staying at the hotel: Meramees Hostel
Hotel price per day: 44.64
Total hotel price for 5 days: 223.2

Plan 1 Recommendations:

Day 1:
Restaurant, Al Refaaee , Price: 39.64
Museums, National Museum of Egyptian Civilization NEMC , Price: 52.61
malls, Mall of Arabia Cairo , Price: 63.15

Day 2:
Restaurant, Khan El Khalili Restaurant , Price: 44.5
Religious Tourism, Mosque of Ibn Tulun , Price: 86.53
Archaeological tourism, Coptic Cairo , Price: 31.94

Day 3:
Restaurant, Shawermer , Price: 33.02
games, Trapped Egypt , Price: 36.13
Tours, Cairo-Overnight Tours , Price: 118.02

Day 4:
Restaurant, TBS- The Bakery Shop , Price: 44.5
Museums, The Coptic Museum , Price: 75.99
Religious Tourism, Mosque of Syeda Zainab , Price: 37.61

Day 5:
Restaurant, Bab El-Sharq , Price: 159.1
Tours, Excursion Cairo Day Tour , Price: 114.23
Religious Tourism, Mosque of Amr I